In [2]:
"""
Basic Example
"""

from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI

from dotenv import load_dotenv
load_dotenv()


prompt = ChatPromptTemplate.from_template("{something}은 캠핑장 예약과 관련된 정보이다. 내용을 이해하라")


In [3]:

model = ChatOpenAI(model="gpt-3.5-turbo")

In [4]:
chain = prompt | model

In [5]:
something = """
[양도] 한알캠핑장 2연석 양도합니다(전체 우레탄 싸이트)

1. 캠핑장 명 :  한알캠핑장

​

2. 이름 :  정지은

​

3. 예약내역 :  2월 23~25일

​

4. 예약사이트(URL) :  캠핏

​

5. 가격 : 2박3일 160,000원 (2연석이니 ×2)

​

6. 취소시간 :  

​

7. 특이사항 : 

​

한알캠핑장은 전면 우레탄으로 되있는

추위 비 눈 걱정없는 캠핑장입니다

가평이고요

​

2월 23~25일 2자리 연석 양도합니다

​

키즈존 3번 4번 한번에 양도합니다

​

한알 이미 여러번간 단골캠인데

친언니와 같이 가고싶어 바로옆자리로 잡았었어요

근데 유치원방학이 한주 미뤄져서

캠가는주랑 겹치게되어 양도내놓습니다

그 다음주로 교환할까 하다가 복잡할까봐

그냥 양도합니다ㅜ

양도양수 이뤄지게되면 반품 취소 불가합니다,
"""

In [6]:
"""
Attaching Function Call information
"""

from langchain.output_parsers.openai_functions import JsonOutputFunctionsParser
functions = [
    {
        "name": "get_camping_info",
        "description": "extract camping info",
        "parameters": {
            "type": "object",
            "properties": {
                "camping_site_name": {
                    "type": "string", 
                    "description": "캠핑장이름, 캠핑장명"
                },
                "reservation_name": {
                    "type": "string",
                    "description": "캠핑장 예약자 이름",
                },
                "reservation_time": {
                    "type": "string",
                    "description": "YYYY-MM-DD format",
                },
                "price": {
                    "type": "number",
                    "description": "캠핑장 가격",
                },
                "special_note": {
                    "type": "string",
                    "description": "특이사항, 참고할 만한 사항",
                },
            },
            "required": ["camping_site_name", "reservation_name", "reservation_time", "price", "special_note"],
        },
    }
]



In [9]:
chain = prompt | model.bind(function_call={"name": "get_camping_info"}, functions=functions) | JsonOutputFunctionsParser()
# chain = prompt | model.bind(function_call={"name": "get_camping_info"}, functions=functions)

In [10]:
chain.invoke({"something": something})

{'camping_site_name': '한알캠핑장',
 'reservation_name': '정지은',
 'reservation_time': '2022-02-23',
 'price': 160000,
 'special_note': '한알캠핑장은 전면 우레탄으로 되있는 추위 비 눈 걱정없는 캠핑장입니다. 가평이고요.'}